In [5]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, Conv2DTranspose
from keras.optimizers import SGD
from skimage import io, color, exposure, transform
import numpy as np
import os
import cv2

In [3]:
from keras import backend as K
K.set_image_dim_ordering('tf')
'''
image_dim_ordering in 'th' mode the channels dimension (the depth) is at index 1 (e.g. 3, 256, 256). " \
"In 'tf' mode is it at index 3 (e.g. 256, 256, 3).
'''

'\nimage_dim_ordering in \'th\' mode the channels dimension (the depth) is at index 1 (e.g. 3, 256, 256). " "In \'tf\' mode is it at index 3 (e.g. 256, 256, 3).\n'

In [10]:
model = Sequential()
model.add(ZeroPadding2D((1, 1), input_shape=(256, 256, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu', name='block1_conv1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(64, (3, 3), activation='relu', name='block1_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, (3, 3), activation='relu', name='block2_conv1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, (3, 3), activation='relu', name='block2_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, (3, 3), activation='relu', name='block3_conv1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, (3, 3), activation='relu', name='block3_conv2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, (3, 3), activation='relu', name='block3_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu', name='block4_conv1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu', name='block4_conv2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu', name='block4_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu', name='block5_conv1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu', name='block5_conv2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu', name='block5_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))

model.load_weights('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', by_name=True)

for layer in model.layers:
    layer.trainable = False
    
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(4096, (7, 7), activation='relu', name='block6_conv1'))

model.add(ZeroPadding2D((1, 1)))
model.add(Conv2DTranspose(512, (1, 1), activation='relu', name='block7_deconv6'))

model.add(ZeroPadding2D((1, 1)))
model.add(Conv2DTranspose(512, (5, 5), activation='relu', name='block7_deconv5'))

model.add(ZeroPadding2D((1, 1)))
model.add(Conv2DTranspose(256, (5, 5), activation='relu', name='block7_deconv4'))

model.add(ZeroPadding2D((1, 1)))
model.add(Conv2DTranspose(128, (5, 5), activation='relu', name='block7_deconv3'))

model.add(ZeroPadding2D((1, 1)))
model.add(Conv2DTranspose(64, (5, 5), activation='relu', name='block7_deconv2'))

model.add(ZeroPadding2D((1, 1)))
model.add(Conv2DTranspose(64, (5, 5), activation='relu', name='block7_deconv1'))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(1, (5, 5), activation='sigmoid', name='pred_alpha'))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='mean_squared_error')